<a href="https://colab.research.google.com/github/SarahGuo97/Fixed-Income-Final-Project/blob/main/Fall_2020_Final_Exam_Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sarah Guo, yg2060@nyu.edu**

2. There exist a Future contract on 5 year government bonds maturing in period 
Deliverable against this Future contract are all government bonds with 4 or 5 year to maturity.
Assume the reference bond for this Future contract is an exact 5 year 6% coupon bond.
The following bonds are eligible for delivery against this Future contract (seller choice)\
a ) 0.50%, 4 period maturity government bond.\
b ) 1.25%, 4 period maturity government bond.\
c ) 2.00%, 4 period maturity government bond.\
d ) 0.75%, 5 period maturity government bond.\
e ) 2.25%, 5 period maturity government bond.

In [40]:
import numpy as np

In [41]:
#risk free rate
def risk_free(n=5, u=0.0015, r0=0.02):
    rf = np.zeros((n+1,n+1))
    rf[n,0] = r0
    for j in range(1, n+1):
      rf[n,j] = rf[n,j-1] + u
      for i in range(n-1,n-j-1,-1):
        rf[i,j] = rf[i+1,j] - u*2
    return 1+rf[:,:n]

rf=risk_free()
rf

array([[1.    , 1.    , 1.    , 1.    , 1.    ],
       [1.    , 1.    , 1.    , 1.    , 1.014 ],
       [1.    , 1.    , 1.    , 1.0155, 1.017 ],
       [1.    , 1.    , 1.017 , 1.0185, 1.02  ],
       [1.    , 1.0185, 1.02  , 1.0215, 1.023 ],
       [1.02  , 1.0215, 1.023 , 1.0245, 1.026 ]])

In [42]:
def calculate_zerobond_price(T, p, fv=1, u=0.0015, r0=0.02):
    # use an price array to hold price in each state along the way
    state_price_array = np.zeros((T+1, T+1))
    
    rf = risk_free(T, u, r0)
    FV = [fv] * (T+1)
    state_price_array[:,-1] = FV # initialize
    # backward
    for i in range(T-1, -1, -1):
        for j in range(0,i+1):
            FV[j] = p*FV[j+1]/rf[T-j,i] + (1-p)*FV[j]/rf[T-j,i]
        state_price_array[T:(T-i-1):-1,i] = FV[0:(i+1)]
    return (FV[0], state_price_array)
calculate_zerobond_price(3,0.6,fv=106)

(99.97519326561692,
 array([[  0.        ,   0.        ,   0.        , 106.        ],
        [  0.        ,   0.        , 104.22812193, 106.        ],
        [  0.        , 102.21453177, 103.92156863, 106.        ],
        [ 99.97519327, 101.61494517, 103.61681329, 106.        ]]))

In [43]:
# this functions returns both bond price at time=0 and a array containing all bond prices along the tree
def calculate_couponbond_price(payment_array, p, u=0.0015, r0=0.02):
    price = 0
    T = len(payment_array)
    # state_price_arrays = []
    state_price_array = np.zeros((T+1,T+1))
    for i in range(len(payment_array)):
        result = calculate_zerobond_price(i+1,p,fv=payment_array[i], u=u, r0=r0)
        price += result[0]
        if i == 0:
            state_price_array[T,0] += result[1][1,0]
        else:
            state_price_array[T-i:T+1,0:i+1] += result[1][1:i+2,0:i+1]
        state_price_array[:,-1] = payment_array[-1]
    return (price, state_price_array)
calculate_couponbond_price([6,6,106],0.6)


(111.62626755003194,
 array([[  0.        ,   0.        ,   0.        , 106.        ],
        [  0.        ,   0.        , 104.22812193, 106.        ],
        [  0.        , 108.10554797, 103.92156863, 106.        ],
        [111.62626755, 107.4886603 , 103.61681329, 106.        ]]))

#### A ) Find period “0” prices of all of the eligible bonds & the reference bond

In [44]:
ref = calculate_couponbond_price([6,6,6,6,106],0.6)[0]
bondA = calculate_couponbond_price([0.5,0.5,0.5,100.5],0.6)[0]
bondB = calculate_couponbond_price([1.25,1.25,1.25,101.25],0.6)[0]
bondC = calculate_couponbond_price([2,2,2,102],0.6)[0]
bondD = calculate_couponbond_price([0.75,0.75,0.75,0.75,100.75],0.6)[0]
bondE = calculate_couponbond_price([2.25,2.25,2.25,2.25,102.25,],0.6)[0]

In [45]:
ref

119.15741171207964

In [46]:
bondA

94.45494888525512

In [47]:
bondB

97.31282754461388

In [48]:
bondC

100.17070620397264

In [49]:
bondD

94.3829638532159

In [50]:
bondE

101.46137752717698

#### B) Find period “0” risk-adjusted or risk-neutral expected prices of period “2” eligible bonds & the reference bond . 𝐸0[𝑃𝑖(2, 𝑠2)]

In [51]:
# discount all cash flows by setting the current node at T=2
EP2ref = calculate_couponbond_price([6,6,106],0.6,0.017)[0]
EP2bondA = calculate_couponbond_price([0.5,100.5],0.6,0.017)[0]
EP2bondB = calculate_couponbond_price([1.25,101.25],0.6,0.017)[0]
EP2bondC = calculate_couponbond_price([2,102],0.6,0.017)[0]
EP2bondD = calculate_couponbond_price([0.75,0.75,100.75],0.6,0.017)[0]
EP2bondE = calculate_couponbond_price([2.25,2.25,102.25,],0.6,0.017)[0]

In [52]:
EP2ref

112.67105705606906

In [53]:
EP2bondA

97.4365796855656

In [54]:
EP2bondB

98.89535427789276

In [55]:
EP2bondC

100.35412887021994

In [56]:
EP2bondD

97.4572338091504

In [57]:
EP2bondE

101.80404045112716

#### C ) Find period “0” the conversion factors for all the eligible bonds
We need to equalize the final risk adjusted expected invoice price such that at TIME 0, both side would be indifferent to which asset to delivery.

In [58]:
ConvFactorA = ref/bondA
ConvFactorA

1.261526400875335

In [59]:
ConvFactorB = ref/bondB
ConvFactorB

1.224477951351798

In [60]:
ConvFactorC = ref/bondC
ConvFactorC

1.1895434925800095

In [61]:
ConvFactorD = ref/bondD
ConvFactorD

1.2624885556401142

In [62]:
ConvFactorE = ref/bondE
ConvFactorE

1.1744115309311929

#### D ) Find the Future price of this Future contract in Periods 0, 1 and 2 in all state.

In [63]:
# price tree of ref bond
ref_tree = calculate_couponbond_price([6,6,6,6,106],0.6)[1]
ref_tree
# the part of the tree that relates to futures price
ref_tree[3:,:3]

array([[  0.        ,   0.        , 112.56532492],
       [  0.        , 116.04290817, 111.62626755],
       [119.15741171, 114.78703761, 110.69799597]])

In [64]:
# starting node different for each branch of tree
def calculate_futures_price(state, period, c):
  if(period==4):
    if(state==2):
      CF = [100*c,100*(1+c)]
      return 0.36*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.017)[0] + 0.24*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.02)[0] + 0.24*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.02)[0] + 0.16*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.023)[0]
    if(state==1):
      CF = [100*c,100*c,100*(1+c)]
      return 0.6*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.0185)[0] + 0.4*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.0215)[0]
    if(state==0):
      CF = [100*c,100*c,100*c,100*(1+c)]
      return calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.02)[0]
  
  if(period==5):
    if(state==2):
      CF = [100*c,100*c,100*(1+c)]
      return 0.36*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.017)[0] + 0.24*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.02)[0] + 0.24*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.02)[0] + 0.16*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.023)[0]
    if(state==1): 
      CF = [100*c,100*c,100*c,100*(1+c)]
      return 0.6*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.0185)[0] + 0.4*calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.0215)[0]
    if(state==0):
      CF = [100*c,100*c,100*c,100*c,100*(1+c)]
      return calculate_couponbond_price(CF, 0.6, u=0.0015, r0=0.02)[0]

In [65]:
# print out the risk-adjusted bond futures price for each period; e.g. H(0,4,A) means the futures price for 4-period-maturity bond A at time 0
period_list = [4,4,4,5,5]
coupon_list = [0.005, 0.0125, 0.02, 0.0075, 0.0225]
bond_list = ["A", "B", "C", "D", "E"]
for i in range(5):
  for s in range(3):
    print("H(%d,%d,%s) = %.4f" % (s,period_list[i],bond_list[i],calculate_futures_price(s,period_list[i],coupon_list[i])))

H(0,4,A) = 94.4549
H(1,4,A) = 95.8440
H(2,4,A) = 97.2316
H(0,4,B) = 97.3128
H(1,4,B) = 98.0091
H(2,4,B) = 98.6892
H(0,4,C) = 100.1707
H(1,4,C) = 100.1741
H(2,4,C) = 100.1469
H(0,5,D) = 94.3830
H(1,5,D) = 95.5206
H(2,5,D) = 96.6516
H(0,5,E) = 101.4614
H(1,5,E) = 101.2406
H(2,5,E) = 100.9842


#### E ) Identify the cheapest to delivery bond in periods 1 and 2 in all states

In [66]:
cheapest = {}
ConvFactor = [ConvFactorA, ConvFactorB, ConvFactorC, 
              ConvFactorD, ConvFactorE]
for i in range(5):
  for s in range(1,3):
    cheapest[(bond_list[i],s)] = calculate_futures_price(s,period_list[i],coupon_list[i])*ConvFactor[i]
    print("H(%d,%d,%s) = %.4f" % (s,period_list[i],bond_list[i],calculate_futures_price(s,period_list[i],coupon_list[i])*ConvFactor[i]))

for k,v in cheapest.items():
  if (v == min(cheapest.values())):
    print(k,v)

H(1,4,A) = 120.9098
H(2,4,A) = 122.6602
H(1,4,B) = 120.0100
H(2,4,B) = 120.8428
H(1,4,C) = 119.1615
H(2,4,C) = 119.1291
H(1,5,D) = 120.5937
H(2,5,D) = 122.0215
H(1,5,E) = 118.8981
H(2,5,E) = 118.5970
('E', 2) 118.59702738010283


Cheapest to delivery bond is Bond E at T=2

#### F ) You sold this Future contract, find a replicating portfolio using bonds “a” through “e” to hedge this future contract in period 0.
Using a ZCB and underlying bond E to form a replicating portfolio:

M(0)\*B(1,u) + N1(0)\*P(1,5,u) = P(1,5,u) - H(0,5)\
M(0)\*B(1,d) + N1(0)\*P(1,5,d) = P(1,5,d) - H(0,5)

M(0)\*1.0185 + N1(0)\*101.7013166 = 101.7013166 - 101.4614\
M(0)\*1.0215 + N1(0)\*100.5495378 = 100.5495378 - 101.4614

    Solve for M(0) = -79.05684489, N1(0) = 0.794083261


